# Goals

* Check Chris' scBaseCount h5ad files downloaded to Chimera

In [1]:
import os
import sys
from pathlib import Path
from glob import glob
from dotenv import load_dotenv
import pandas as pd

In [2]:
load_dotenv(override=True)

True

In [3]:
# number of rows in the srx_metadata table
pd.options.display.max_rows = 4

In [4]:
# get the base directory of the repo
base_dir = !git rev-parse --show-toplevel
base_dir = Path(base_dir[0])

In [5]:
# add scripts/sql_db_utils.py to the path
import sys
sys.path.append(str(base_dir / 'scripts'))
from sql_db_utils import db_connect, get_srx_metadata_for_star_results

In [6]:
with db_connect() as conn:
    star_meta = get_srx_metadata_for_star_results(conn)
star_meta

,database,entrez_id,srx_accession,is_illumina,is_single_cell,is_paired_end,lib_prep,tech_10x,cell_prep,organism,tissue,disease,perturbation,cell_line,czi_collection_id,czi_collection_name,notes,created_at,updated_at,tissue_ontology_term_id
0,sra,30468622,SRX22482811,yes,yes,yes,10x_Genomics,5_prime_gex,single_cell,Homo sapiens,blood,not specified,not specified,CD4+ T Cells,None,None,Metadata obtained by SRAgent,2025-01-23 07:55:25.498733,2025-06-04 07:01:33.825488,UBERON:0000178
1,sra,19246423,SRX13824065,yes,yes,yes,10x_Genomics,3_prime_gex,single_cell,Mus musculus,skin,unsure,unsure,unsure,None,None,Metadata obtained by SRAgent,2025-04-08 23:15:43.049133,2025-04-08 23:17:45.052816,UBERON:0022295
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59553,sra,37791063,SRX28094413,yes,yes,yes,10x_Genomics,3_prime_gex,single_cell,Mus musculus,duodenal epithelium,none (helminth infection model),"Heligmosomoides polygyrus infection, 7 days po...",none,None,None,Metadata obtained by SRAgent,2025-06-26 16:50:31.422198,2025-06-26 16:52:13.114066,UBERON:0008346
59554,sra,37821277,SRX28122833,yes,yes,yes,10x_Genomics,other,single_cell,Mus musculus,primary chondrocytes,,,,None,None,Metadata obtained by SRAgent,2025-06-26 16:50:31.422198,2025-06-26 16:53:03.743177,UBERON:0000469


In [7]:
# Chris' h5ad files
file_path = Path('/large_storage/ctc/public/scBasecamp/GeneFull_Ex50pAS/GeneFull_Ex50pAS')

In [8]:
# find all h5ad files in the directory
h5ad_files = {}
for x in glob(str(file_path / '**' / '*.h5ad')):
    # basename lacking extension
    accession = Path(x).name.split('.')[0]
    h5ad_files[accession] = x

# number of files
len(h5ad_files)

61190

In [12]:
# intersect of star_meta["srx_accession"] and h5ad_files.keys()
x = star_meta["srx_accession"].isin(list(h5ad_files.keys()))
star_meta_intersect = star_meta[x]
star_meta_intersect

,database,entrez_id,srx_accession,is_illumina,is_single_cell,is_paired_end,lib_prep,tech_10x,cell_prep,organism,tissue,disease,perturbation,cell_line,czi_collection_id,czi_collection_name,notes,created_at,updated_at,tissue_ontology_term_id
0,sra,30468622,SRX22482811,yes,yes,yes,10x_Genomics,5_prime_gex,single_cell,Homo sapiens,blood,not specified,not specified,CD4+ T Cells,None,None,Metadata obtained by SRAgent,2025-01-23 07:55:25.498733,2025-06-04 07:01:33.825488,UBERON:0000178
1,sra,19246423,SRX13824065,yes,yes,yes,10x_Genomics,3_prime_gex,single_cell,Mus musculus,skin,unsure,unsure,unsure,None,None,Metadata obtained by SRAgent,2025-04-08 23:15:43.049133,2025-04-08 23:17:45.052816,UBERON:0022295
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59501,sra,33136781,SRX24811443,yes,yes,yes,10x_Genomics,5_prime_gex,single_cell,Homo sapiens,CD8+ T cells,unsure,Cells were activated using an optimized Stim‐R...,unsure,None,None,Metadata obtained by SRAgent,2025-04-06 16:00:38.940032,2025-06-25 15:20:58.119513,
59502,sra,35101897,SRX26008878,yes,yes,yes,10x_Genomics,3_prime_gex,single_cell,Homo sapiens,hiPSC,autism spectrum disorder (ASD) and neurodevelo...,CRISPR-based gene activation (CRISPRa),WTC11,None,None,Metadata obtained by SRAgent,2025-01-21 13:58:24.384458,2025-06-25 15:28:48.022424,


In [15]:
star_meta_missing = star_meta[~x].reset_index(drop=True)
star_meta_missing

,database,entrez_id,srx_accession,is_illumina,is_single_cell,is_paired_end,lib_prep,tech_10x,cell_prep,organism,tissue,disease,perturbation,cell_line,czi_collection_id,czi_collection_name,notes,created_at,updated_at,tissue_ontology_term_id
0,sra,27685359,SRX20264913,yes,yes,yes,10x_Genomics,3_prime_gex,single_cell,Danio rerio,border zone and injured ventricle tissues,cardiac regeneration study,"cd74a; cd74b mutant, immune response investiga...",not applicable,None,None,Metadata obtained by SRAgent,2025-02-20 12:00:48.206497,2025-02-20 12:02:02.507143,None
1,sra,37698231,ERX13269362,yes,yes,yes,10x_Genomics,3_prime_gex,single_cell,Mus musculus,adipose tissue,none,none,none,None,None,Metadata obtained by SRAgent,2025-04-09 13:05:35.586718,2025-04-09 13:07:43.632407,UBERON:0001013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,sra,37791063,SRX28094413,yes,yes,yes,10x_Genomics,3_prime_gex,single_cell,Mus musculus,duodenal epithelium,none (helminth infection model),"Heligmosomoides polygyrus infection, 7 days po...",none,None,None,Metadata obtained by SRAgent,2025-06-26 16:50:31.422198,2025-06-26 16:52:13.114066,UBERON:0008346
668,sra,37821277,SRX28122833,yes,yes,yes,10x_Genomics,other,single_cell,Mus musculus,primary chondrocytes,,,,None,None,Metadata obtained by SRAgent,2025-06-26 16:50:31.422198,2025-06-26 16:53:03.743177,UBERON:0000469


In [22]:
# write star_meta_intersect to a csv
star_meta_intersect.to_csv(base_dir / "data" / "star-meta-present_h5ad-missing.csv", index=False)

In [21]:
h5ad_files_only = list(set(h5ad_files.keys()) - set(star_meta["srx_accession"].tolist()))
len(h5ad_files_only)

2304